[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show2d/show2d_all_features.ipynb)

# Show2D — All Features

Comprehensive demo of every Show2D feature using realistic electron microscopy synthetic data:
HRTEM lattice fringes, HAADF-STEM atomic columns, SAED diffraction patterns, focal series, and more.

In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [2]:
import numpy as np
from quantem.widget import Show2D
from IPython.display import display

np.random.seed(42)


def make_hrtem(size=256):
    """Simulate HRTEM lattice fringes (like Si [110])."""
    y, x = np.mgrid[:size, :size]
    img = np.zeros((size, size))
    freqs = [(0.08, 0.0), (0.06, np.pi / 3), (0.10, np.pi / 6)]
    for freq, angle in freqs:
        img += np.cos(2 * np.pi * freq * (x * np.cos(angle) + y * np.sin(angle)))
    envelope = 1.0 / (1 + np.exp(-0.05 * (min(size // 2, 100) - np.sqrt((x - size // 2)**2 + (y - size // 2)**2))))
    img = img * envelope
    img += np.random.normal(0, 0.3, (size, size))
    return img.astype(np.float32)


def make_haadf(size=256, spacing=16, sigma=2.5):
    """Simulate HAADF-STEM image with atomic columns on a hexagonal lattice."""
    y, x = np.mgrid[:size, :size]
    img = np.random.normal(0.1, 0.02, (size, size))
    a1 = np.array([spacing, 0])
    a2 = np.array([spacing / 2, spacing * np.sqrt(3) / 2])
    for i in range(-1, size // spacing + 2):
        for j in range(-1, size // spacing + 2):
            cx = i * a1[0] + j * a2[0]
            cy = i * a1[1] + j * a2[1]
            if -spacing < cx < size + spacing and -spacing < cy < size + spacing:
                intensity = 0.8 + 0.2 * np.random.rand()
                img += intensity * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    return img.astype(np.float32)


def make_diffraction(size=256, n_rings=4):
    """Simulate SAED diffraction pattern with Bragg spots."""
    y, x = np.mgrid[:size, :size]
    cx, cy = size // 2, size // 2
    pattern = 8.0 * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * 4**2))
    for ring in range(1, n_rings + 1):
        r = ring * 22
        n_spots = 6
        for k in range(n_spots):
            angle = k * 2 * np.pi / n_spots + ring * np.pi / 12
            sx = cx + r * np.cos(angle)
            sy = cy + r * np.sin(angle)
            intensity = 2.0 / ring
            spot_sigma = max(1.5, 3.0 - ring * 0.3)
            pattern += intensity * np.exp(-((x - sx)**2 + (y - sy)**2) / (2 * spot_sigma**2))
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)
    pattern += 0.3 * np.exp(-dist / 60)
    pattern = np.maximum(pattern, 0)
    pattern = np.random.poisson(np.clip(pattern * 100, 0, 1e6)).astype(np.float32) / 100
    return pattern


print("Generator functions ready.")

Generator functions ready.


## 1. Single HRTEM Image

Crystal lattice fringes with amorphous edge envelope and shot noise.

In [3]:
hrtem = make_hrtem(256)
Show2D(hrtem, title="HRTEM — Si [110] Lattice Fringes", cmap="gray")

## 2. Single HAADF-STEM Image

Atomic columns arranged on a hexagonal lattice with intensity variations simulating Z-contrast.

In [4]:
haadf = make_haadf(256, spacing=20, sigma=2.5)
Show2D(haadf, title="HAADF-STEM — Hexagonal Atomic Columns", cmap="inferno")

## 3. Gallery with Labels

Six different EM image types displayed together.

In [5]:
# HRTEM
img_hrtem = make_hrtem(128)

# HAADF
img_haadf = make_haadf(128, spacing=12, sigma=1.8)

# Diffraction pattern
img_diff = make_diffraction(128, n_rings=3)

# Phase contrast — cosine of HRTEM (simulates CTF-filtered phase image)
img_phase = np.cos(make_hrtem(128) * np.pi).astype(np.float32)

# Dark field — select one lattice frequency
y, x = np.mgrid[:128, :128]
lattice = np.cos(2 * np.pi * 0.08 * x)
envelope = 1.0 / (1 + np.exp(-0.05 * (50 - np.sqrt((x - 64)**2 + (y - 64)**2))))
img_darkfield = (np.abs(lattice) * envelope + np.random.normal(0, 0.1, (128, 128))).astype(np.float32)

# Strain map — gradient of lattice displacement
displacement = np.sin(2 * np.pi * x / 128) * np.sin(2 * np.pi * y / 128) * 3.0
strain_xx = np.gradient(displacement, axis=1).astype(np.float32)

gallery_images = [img_hrtem, img_haadf, img_diff, img_phase, img_darkfield, strain_xx]
gallery_labels = ["HRTEM", "HAADF", "Diffraction", "Phase Contrast", "Dark Field", "Strain (exx)"]

Show2D(gallery_images, labels=gallery_labels, title="EM Image Gallery", ncols=3)

## 4. Gallery from 3D Array — Focal Series

Simulated focal series: the same crystal structure at different defocus values,
modeled by varying the CTF envelope width.

In [6]:
n_focal = 6
size = 128
y, x = np.mgrid[:size, :size]

# Base lattice with multiple periodicities
base_lattice = (
    np.cos(2 * np.pi * 0.08 * x)
    + np.cos(2 * np.pi * 0.06 * (x * np.cos(np.pi / 3) + y * np.sin(np.pi / 3)))
)

focal_series = np.zeros((n_focal, size, size), dtype=np.float32)
defocus_values = np.linspace(-60, 60, n_focal)

for i, defocus in enumerate(defocus_values):
    # CTF-like transfer: defocus changes envelope width
    sigma_env = max(20, 60 - abs(defocus) * 0.5)
    env = np.exp(-((x - size // 2)**2 + (y - size // 2)**2) / (2 * sigma_env**2))
    # Phase shift from defocus
    phase_shift = defocus * 0.02
    img = base_lattice * np.cos(phase_shift) * env
    img += np.random.normal(0, 0.2, (size, size))
    focal_series[i] = img.astype(np.float32)

focal_labels = [f"Defocus {d:.0f} nm" for d in defocus_values]
Show2D(focal_series, labels=focal_labels, title="Focal Series", ncols=3, cmap="gray")

## 5. Different-sized Images

Images from different detectors with varying pixel counts. Show2D resizes them to a common size.

In [7]:
img_128 = make_haadf(128, spacing=10, sigma=1.5)
img_256 = make_hrtem(256)
img_200 = make_diffraction(200, n_rings=3)

Show2D(
    [img_128, img_256, img_200],
    labels=["HAADF 128x128", "HRTEM 256x256", "Diffraction 200x200"],
    title="Mixed Detector Sizes",
    ncols=3,
)

## 6. Colormaps

The same HAADF image under all available colormaps.

In [8]:
haadf_cmap = make_haadf(192, spacing=18, sigma=2.2)

for cmap in ["inferno", "viridis", "magma", "plasma", "gray"]:
    display(Show2D(haadf_cmap, title=f"HAADF — {cmap}", cmap=cmap))

## 7. FFT — Single Image

HRTEM lattice fringes with FFT enabled. The reciprocal lattice spots should be visible
at spatial frequencies corresponding to the real-space lattice periodicities.

In [9]:
hrtem_fft = make_hrtem(256)
Show2D(hrtem_fft, title="HRTEM + FFT", cmap="gray", show_fft=True)

## 8. FFT — Gallery

Multiple lattice orientations with different spatial frequencies.
Click each to see its FFT showing different reciprocal lattice spots.

In [10]:
size = 192
y, x = np.mgrid[:size, :size]

fft_gallery = []
fft_labels = []
orientations = [
    (0.05, 0.0, "Horizontal 0.05"),
    (0.08, np.pi / 4, "45 deg 0.08"),
    (0.12, np.pi / 6, "30 deg 0.12"),
    (0.06, np.pi / 2, "Vertical 0.06"),
]

for freq, angle, label in orientations:
    lattice = np.cos(2 * np.pi * freq * (x * np.cos(angle) + y * np.sin(angle)))
    lattice += np.random.normal(0, 0.2, (size, size))
    fft_gallery.append(lattice.astype(np.float32))
    fft_labels.append(label)

Show2D(fft_gallery, labels=fft_labels, title="Lattice Orientations + FFT", show_fft=True, cmap="gray", ncols=2)

## 9. Log Scale

Diffraction pattern with log scale — the pattern naturally spans several orders of magnitude
from the intense central beam to weak high-order reflections.

In [11]:
diff_log = make_diffraction(256, n_rings=5)

display(Show2D(diff_log, title="Diffraction — Linear Scale", cmap="inferno"))
display(Show2D(diff_log, title="Diffraction — Log Scale", cmap="inferno", log_scale=True))

## 10. Auto Contrast

HAADF with one extremely bright outlier atom column.
Auto contrast uses percentiles to clip the display range so the rest of the image is visible.

In [12]:
haadf_outlier = make_haadf(256, spacing=20, sigma=2.5)
# Add one very bright outlier column
oy, ox = np.mgrid[:256, :256]
haadf_outlier += 50.0 * np.exp(-((ox - 128)**2 + (oy - 100)**2) / (2 * 3**2))

display(Show2D(haadf_outlier, title="HAADF with Outlier — No Auto Contrast"))
display(Show2D(haadf_outlier, title="HAADF with Outlier — Auto Contrast", auto_contrast=True))

## 11. Scale Bar

HRTEM with a realistic pixel size. At 0.05 nm/px (0.5 angstrom/px), the scale bar
displays physical dimensions.

In [13]:
hrtem_scale = make_hrtem(512)
Show2D(
    hrtem_scale,
    title="HRTEM with Scale Bar (0.5 A/px)",
    cmap="gray",
    pixel_size_angstrom=0.5,
)

## 12. Custom Columns

Gallery of 8 simulated SAED patterns at different zone axes,
displayed in 4 columns.

In [14]:
zone_axes = [
    ("[001]", 3, 0.0),
    ("[110]", 4, np.pi / 12),
    ("[111]", 3, np.pi / 6),
    ("[112]", 5, np.pi / 8),
    ("[011]", 4, np.pi / 4),
    ("[012]", 3, np.pi / 3),
    ("[113]", 5, np.pi / 5),
    ("[102]", 4, np.pi / 10),
]

saed_images = []
saed_labels = []
for label, n_rings, rot in zone_axes:
    size = 128
    y, x = np.mgrid[:size, :size]
    cx, cy = size // 2, size // 2
    pattern = 8.0 * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * 4**2))
    for ring in range(1, n_rings + 1):
        r = ring * 18
        for k in range(6):
            angle = k * 2 * np.pi / 6 + rot + ring * np.pi / 12
            sx = cx + r * np.cos(angle)
            sy = cy + r * np.sin(angle)
            intensity = 2.0 / ring
            spot_sigma = max(1.5, 3.0 - ring * 0.3)
            pattern += intensity * np.exp(-((x - sx)**2 + (y - sy)**2) / (2 * spot_sigma**2))
    dist = np.sqrt((x - cx)**2 + (y - cy)**2)
    pattern += 0.3 * np.exp(-dist / 50)
    pattern = np.maximum(pattern, 0)
    pattern = np.random.poisson(np.clip(pattern * 100, 0, 1e6)).astype(np.float32) / 100
    saed_images.append(pattern)
    saed_labels.append(label)

Show2D(
    saed_images,
    labels=saed_labels,
    title="SAED Patterns at Different Zone Axes",
    ncols=4,
    log_scale=True,
)

## 13. Hide Controls & Stats

Minimal view — no UI controls or statistics, just the image.

In [15]:
hrtem_minimal = make_hrtem(256)
Show2D(
    hrtem_minimal,
    title="HRTEM — Minimal View",
    cmap="gray",
    show_controls=False,
    show_stats=False,
)

## 14. Custom Panel & Image Size with FFT

Custom sizing: larger FFT/histogram panel and specified image width.

In [16]:
hrtem_custom = make_hrtem(256)
Show2D(
    hrtem_custom,
    title="HRTEM — Custom Sizes + FFT",
    cmap="gray",
    show_fft=True,
    panel_size_px=200,
    image_width_px=400,
)